
# Scoring Models

The following notebook will demonstrate how to use Earth-2 MIP to perform a scoring
workflow to assess the accuracy of AI models using ERA5 reanalysis data as the
ground truth. This can then be extended to score custom models placed into the model
registry. Usually, you'd be using file from disk, eg a from a HDF5 datasource. The expected
format of this data would be a file containing a full year worth of data, which would be too
large to ship for the purpose os a workshop. We will therefore use the CDS datasource
accessing cached files.

In summary this notebook will cover the following topics:

- Implementing a basic scoring workflow in Earth-2 MIP
- HDF5 datasource and the expected data format of the H5 files


In [ ]:
import datetime
import os

## Setting up HDF5 data
The first step of scoring is handling the target data. One could simply use the
CDSDatasource to download target data on the fly, but depending on how comprehensive
the scoring is, this can prove quite slow.
Additionally, many scoring pipelines require on-prem data.
Thus, this will demonstrate how to use the HDF5 datasource.
The HDF5 data source assumes that the data to be loaded is stored in the general form:

-  year.h5
      | - field (time, channels, grid)

For DLWP which requires 7 channels with a time-step size of 12 hours, an H5 file will
have the following form of data for an entire year:

-  2017.h5
      | - field (730, 7, 720, 1440)
-  2016.h5
      | - field (730, 7, 720, 1440)
-  2015.h5
      | - field (730, 7, 720, 1440)

<div class="alert alert-info"><h4>Note</h4><p>There is some flexibility with the dimensions of the data in the H5 files. The time
  dimension may be some factor of 12 (such as 6hr dt or 4hr dt) and the fields may
  contain additional channels not needed by the model. The data source will select the
  necessary data for the model. Additionally, the later two dims have some flexibility
  with regridding.</p></div>

One option to build these H5 files from scratch is to use the ERA5 mirror scripts
provided in [Modulus](https://github.com/NVIDIA/modulus/tree/main/examples/weather/dataset_download).
For the rest of this tutorial, it is assumed that 2017.h5 is present for the full year.



## Loading Models
Having discussed the HDF5 data source, the next step is to load our model we wish to score.
In this tutorial we will have cached data for FourCastNet SFNO (fcnv2_sm) and for DLWP.
Take note of the `e2mip://` which will direct Earth-2 MIP to load a known model package.
FourcastNet AFNO (fcn) can be selected as well, note though that this will trigger some
downloads from CDS which might take a while.

In [ ]:
from modulus.distributed import DistributedManager

from earth2mip import registry
from earth2mip.networks import dlwp, fcnv2_sm, fcn

model_name = 'dlwp' # 'dlwp', 'fcnv2_sm', 'fcn'

device = DistributedManager().device
package = registry.get_model(f"e2mip://{model_name}")
if model_name == 'dlwp':
    model = dlwp.load(package, device=device)
elif model_name == 'fcnv2_sm':
    model = fcnv2_sm.load(package, device=device)
elif model_name == 'fcn':
    model = fcn.load(package, device=device)
else:
    raise ValueError(f'model {model_name} not implemented')

## HDF5 Datasource
With the H5 files properly formatted, a H5 datasource could now get defined.
Find some commented code of how this would look like.
Defining the HDF5 data source requires two items: a root directory location of the H5
files as well as some metadata.
The metadata is a JSON/dictionary object that helps Earth-2 MIP index the H5 file.
Typically, this can be done by placing a `data.json` file next to the H5 files.
See [this documentation](https://github.com/NVIDIA/earth2mip/blob/f44c580ccc3d98bf349fe97823bb1540e532c80d/earth2mip/initial_conditions/hdf5.py#L38)
for more details on how to set up input data correctly.



In [ ]:
from earth2mip.initial_conditions import cds

# h5_folder = '/e2ws_data/era5_73var'
# datasource = hdf5.DataSource.from_path(
#     root=h5_folder, channel_names=model.channel_names
# )


# Test to see if our datasource is working
datasource = cds.DataSource(model.in_channel_names)
time = datetime.datetime(2017, 8, 24, 12)
out = datasource[time]
print(out.shape)

## Running Scoring
With the datasource and model loaded, scoring can now be performed.
To score, we will run 4 day forecasts at 30 day intervals.
For research, one would typically want this to be much more comprehensive
so feel free to customize for your use case.
To avoid massively inflating the data requirements we limited this task to
two four day hindcasts, much larger sets can also be analysed in seconds or minutes.

The `score_deterministic` API provides a simple way to calculate RMSE and ACC scores.
ACC scores require climatology which is beyond the scope of this example, thus zero
values will be provided and only the RMSE will be of concern.
This function will save the results of every inference run into a CSV file which
can then be pose process using some of the utility functions Earth-2 MIP provides.


In [ ]:
from earth2mip.inference_medium_range import save_scores, time_average_metrics
import numpy as np

# Use 2 initializations.
time = datetime.datetime(2017, 8, 24, 12)
initial_times = [
    time + datetime.timedelta(days=30 * i, hours=6 * i) for i in range(2)
]  # modify here to change the initializations

# Output directoy, delete old scoring file if present
output_dir = "outputs/02_model_scoring"
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)
if os.path.exists(output_dir + '/0.csv'):
    os.remove(output_dir + '/0.csv')

output = save_scores(
    model,
    n=16,  # 6 hour timesteps (16*6/24 = 4-day forecast)
    initial_times=initial_times,
    data_source=datasource,
    time_mean=np.zeros((len(model.in_channel_names), 721, 1440)),
    output_directory=output_dir,
)

## Post Processing
The last step is any post processing / IO that is desired.
Typically, it is recommended to save the output dataset to a netCDF file for further processing.
Let's plot the RMSE of the z500 (geopotential at pressure level 500) field.

Hint: if you receive `ValueError: cannot convert a DataFrame with a non-unique MultiIndex into xarray`,
go to `output_dir` (should be located under `outputs/02_model_scoring/`), delete the file `0.csv` and 
re-run the inference cell above once.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from earth2mip.forecast_metrics_io import read_metrics

series = read_metrics(output_dir)
dataset = time_average_metrics(series)

plt.close("all")
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
channels = ["z500", "t2m", "t850"]
t = dataset.lead_time / pd.Timedelta("1 h")
for i, channel in enumerate(channels):
    y = dataset.rmse.sel(channel=channel)
    axs[i].plot(t[1:], y[1:])  # Ignore first output as that's just initial condition.
    axs[i].set_xlabel("Lead Time (hours)")
    axs[i].set_ylabel("RMSE")
    axs[i].set_title(f"{model_name} {channel} RMSE 2017")

plt.savefig(f"{output_dir}/dwlp_rmse.png")

This completes this introductory notebook on basic scoring of models in Earth-2 MIP,
which is founational for comparing the performance of different models.

